本次分享主要内容是大模型最核心的地方：自注意力机制。
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image001.png)

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image003.png)

接下来会分四部分，第一部分是自注意力机制简化版，第二部分是自注意力机制，第三部分是因果注意力(causal attention)，第四部分是多头注意力(multi-head attention)。

#### 1.The problem with modeling long sequences
让咱们看下在没有自注意力机制之前如何解决长序列问题的，这里咱们拿语言翻译举例。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image005.png)

上图是德文翻译成英文的例子，发现单词不能逐词翻译，不然就会乱套，因为每种语言的语法不一样的，这需要模型能够看到上下文，才能做出正确的翻译。</br>

那么自然想到的办法是使用encoder-decoder架构，先使用encoder看到所有输入，理解上下文，然后使用decoder逐词输出。</br>

在transformer架构出来之前，最流行的encoder-decoder架构是RNN，比如LSTM和GRU。


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image007.png)

如上图所示，所有的输入先经过RNN encoder，得到一个隐状态，在encoder里，输出是下一步的输入，所以隐状态是包含了上下文的。然后隐状态作为decoder的输入。

可以简单把hidden state理解为包含了上下文的embedding vector。

RNN的缺点是，decoder无法获取之前的隐状态信息，所以需要把所有的隐状态都保存下来，这样会占用大量内存。同时长距离文本处理起来比较困难。



#### 2.Capturing data dependencies with self-attention

2014年，研究人员提出了一个Bahdanau Attention的机制，通过修改encoder-decoder架构，以便decoder能够有选择的获得输入序列的不同部分。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image009.png)

如上图，在decoder阶段，针对特定输出，可以获得输入序列的不同部分，并通过attention weight来衡量输入序列不同部分对当前输出的重要性。

Bahdanau Attention机制的缺点是，无法并行处理，在encoder阶段，所有的输入必须一个一个处理，在decoder阶段，当前时刻的隐状态依赖于前一个时刻的隐状态，所以也无法并行处理。

三年后2017年，受到了Bahdanau Attention的启发，提出了transformer架构，使用自注意力机制来解决长距离依赖问题。

Self-attention（自注意力）是一种让输入序列中的每个位置都能够在计算序列表示时关注同一序列中所有位置的机制。这种机制使得模型能够捕捉序列内部的长距离依赖关系，并且理解不同位置之间的复杂关系。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image011.png)

#### 3.Attending to different parts of the input with self-attention

本章会介绍自注意力机制内部工作机制，以及学习如何从头实现自注意力机制。自注意力机制作为大模型的基石，非常值得花费时间去理解，一旦掌握了机制背后的原理，对后续的深入学习非常有帮助。

self-attention里的self表示什么意思呢？

“Self”在自注意力中指的是机制能够计算不同位置之间的注意力权重，这些位置都位于单个输入序列内部。这意味着模型在处理序列时，会分析序列内部各部分之间的关系，例如句子中的单词或图像中的像素。

自注意力机制评估并学习输入序列内部各部分之间的关系和依赖性。这种机制使得模型能够理解序列中各个元素之间的相互作用，而不仅仅是序列作为一个整体与其他序列之间的关系。

传统的注意力机制通常关注两个不同序列之间的元素关系，例如在序列到序列（sequence-to-sequence）模型中，注意力可能集中在输入序列和输出序列之间的关系。这种注意力机制在机器翻译等任务中很常见，其中模型需要关注输入序列（如一种语言的文本）与输出序列（如另一种语言的文本）之间的关系，比如上图所示。

自注意力在自然语言处理（NLP）和计算机视觉（CV）等领域有广泛应用。在NLP中，它可以用于理解文本中单词之间的关系；在CV中，它可以用于理解图像中不同像素或区域之间的关系。

自注意力机制比较复杂，为了让大家更方便的理解其原理，我们先实现一个简单的没有可训练权重的自注意力机制，然后逐步增加可训练权重，最后实现一个完整的自注意力机制。


##### 3.1 A simple self-attention mechanism without trainable weights

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image013.png)

自注意力机制的目标是为输入序列中的每个元素计算一个上下文向量(context vector),这个向量结合了来自序列中所有其他元素的信息。假如咱们要对第二个元素${x^{(2)}}$计算上下文向量${z^{(2)}}$,每个输入元素对于计算上下文向量${z_2}$的重要性或者贡献度由注意力权重${a_{21}}$到${a_{2T}}$决定。这里的T表示输入序列的长度，而${a_{2i}}$表示第i个元素对第2个元素的注意力权重。上下文向量${z^{(2)}}$的计算公式如下：

$$z^{(2)} = \sum_{i=1}^{T} a_{2i} x^{(i)}$$

为了计算注意力权重，咱们需要一个打分函数，这个函数的作用是衡量两个元素之间的相似度。常用的打分函数是点积，即两个向量对应元素相乘然后求和。

$$e_{ij} = x_i^T x_j$$

In [1]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image015.png)

这里我们将输入元素${x^{(2)}}$叫作query,为什么叫作query，留到下回分解。

我们先来看下中间一步的计算，计算attention score, 即query ${x^{(2)}}$ 与其他输入元素的点积。

点积计算公式如下：
$$e_{ij} = x_i^T x_j$$

也就是两个向量的元素对应相乘然后求和。

In [2]:
query = inputs[1]
atten_scores_2 = torch.empty(inputs.shape[0])
for i in range(inputs.shape[0]):
    atten_scores_2[i] = torch.dot(query, inputs[i])
print(atten_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
res = 0
for idx, element in enumerate(inputs[0]):
    res += element * query[idx]
print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


点积其实是衡量两个向量相似度的方法之一，点积越大，两个向量越相似。这里使用到点积来衡量一个文本序列里，不同token之间的关系，点积越大，关系越密切。
![](./image.png)

下一步就是通过归一化attention score，得到注意力权重attention weight。

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image017.png)

归一化公式如下：
$$a_{ij} = \frac{{w_{ij}}}{\sum_{k=1}^{T} w_{ik}}$$

归一化的目的是让注意力权重之和为1，这样就可以保证所有注意力权重加起来等于1。归一化对可解释性和训练稳定性都有好处。

In [4]:
atten_weights_2_tmp = atten_scores_2 / atten_scores_2.sum()
print("Attention weights:",atten_weights_2_tmp)
print("Sum:",atten_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


在实际工程中，使用softmax函数来做归一化更为常见，这种方法对于极大或者极小的值更为鲁棒，同时还能提供更有利的梯度特性。softmax函数公式如下：

$$a_{ij} = \frac{{e^{w_{ij}}}}{\sum_{k=1}^{T} e^{w_{ik}}}$$

In [5]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
atten_weights_2_naive = softmax_naive(atten_scores_2)
print("Attention weights:",atten_weights_2_naive)
print("Sum:",atten_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


softmax方法可以确保attention weight总是正的，而除以sum的方法无法保证这一点。这点可以让attention weight可以看作概率分布或者相对重要性。

softmax_naive方法的缺点是，对于极大或者极小的值，计算结果不稳定，容易出现数值溢出。因此建议使用PyTorch自带的softmax函数。

In [6]:
atten_weights_2 = torch.softmax(atten_scores_2, dim=0)
print("Attention weights:",atten_weights_2)
print("Sum:",atten_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch03__image019.png)
最后一步就是计算上下文向量context vector。

计算公式如下：

$$z^{(2)} = \sum_{i=1}^{T} a_{2i} x^{(i)}$$

每一个输入向量乘以对应的attention weight，然后求和，得到上下文向量。

In [7]:
query = inputs[1]
context_vector_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vector_2 += atten_weights_2[i] * x_i
print("Context vector:",context_vector_2)

Context vector: tensor([0.4419, 0.6515, 0.5683])
